---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [23]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    from sklearn.model_selection import train_test_split
    from sklearn.neural_network import MLPClassifier
    from sklearn.preprocessing import MinMaxScaler

    df_train = pd.read_csv('train.csv', encoding = "ISO-8859-1")
    df_test = pd.read_csv('test.csv', encoding = "ISO-8859-1")


    list_to_remove = ['balance_due',
     'collection_status',
     'compliance_detail',
     'payment_amount',
     'payment_date',
     'payment_status']

    list_to_remove_all = ['violator_name', 'zip_code', 'country', 'city',
                          'inspector_name', 'violation_street_number', 'violation_street_name',
                          'violation_zip_code', 'violation_description',
                          'mailing_address_str_number', 'mailing_address_str_name',
                          'non_us_str_code',
                          'ticket_issued_date', 'hearing_date', 'admin_fee']

    df_train.drop(list_to_remove, axis=1, inplace=True)
    df_train.drop(list_to_remove_all, axis=1, inplace=True)
    df_test.drop(list_to_remove_all, axis=1, inplace=True)

    df_train.drop('grafitti_status', axis=1, inplace=True)
    df_test.drop('grafitti_status', axis=1, inplace=True)

    ###
    df_latlons = pd.read_csv('latlons.csv')
    df_address =  pd.read_csv('addresses.csv')
    df_id_latlons = df_address.set_index('address').join(df_latlons.set_index('address'))

    df_train = df_train.set_index('ticket_id').join(df_id_latlons.set_index('ticket_id'))
    df_test = df_test.set_index('ticket_id').join(df_id_latlons.set_index('ticket_id'))

    ###
    vio_code_freq10 = df_train.violation_code.value_counts().index[0:10]
    df_train['violation_code_freq10'] = [list(vio_code_freq10).index(c) if c in vio_code_freq10 else -1 for c in df_train.violation_code ]
    df_train.drop('violation_code', axis=1, inplace=True)
    df_test['violation_code_freq10'] = [list(vio_code_freq10).index(c) if c in vio_code_freq10 else -1 for c in df_test.violation_code ]
    df_test.drop('violation_code', axis=1, inplace=True)

    ###
    df_train = df_train[df_train.compliance.isnull() == False]

    df_train.lat.fillna(method='pad', inplace=True)
    df_train.lon.fillna(method='pad', inplace=True)
    df_train.state.fillna(method='pad', inplace=True)

    df_test.lat.fillna(method='pad', inplace=True)
    df_test.lon.fillna(method='pad', inplace=True)
    df_test.state.fillna(method='pad', inplace=True)

    one_hot_encode_columns = ['agency_name', 'state', 'disposition', 'violation_code_freq10']

    df_train = pd.get_dummies(df_train, columns=one_hot_encode_columns)
    df_test = pd.get_dummies(df_test, columns=one_hot_encode_columns)
        
    ## Resampling:
    df_train.sort_values('compliance', ascending=False, inplace=True)
    num_of_com = int(df_train.compliance.sum())
    df_train = df_train.append([df_train[0:num_of_com]]*10, ignore_index=True)
    ## End of Resampling

    train_features = df_train.columns.drop('compliance')

    #######
    X_train = df_train[train_features]
    y_train = df_train.compliance

    X_test = df_test
    l1 = list(set(X_test.columns) - set(X_train.columns))
    X_test.drop(l1, axis=1, inplace=True)

    l2 = list(set(X_train.columns) - set(X_test.columns))
    X_train = X_train.drop(l2, axis=1)

    #######

    scaler = MinMaxScaler()

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    
    clf = GradientBoostingClassifier()
    clf.fit(X_train_scaled, y_train)

    test_proba = clf.predict_proba(X_test_scaled)[:,1]
    test_df = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    test_df['compliance'] = test_proba
    test_df.set_index('ticket_id', inplace=True)
    
    return test_df.compliance

In [24]:
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.331487
285362    0.127701
285361    0.393810
285338    0.452095
285346    0.508352
285345    0.452095
285347    0.480539
285342    0.968860
285530    0.107760
284989    0.240953
285344    0.450086
285343    0.159732
285340    0.162459
285341    0.455085
285349    0.508352
285348    0.452095
284991    0.240953
285532    0.192581
285406    0.169085
285001    0.319854
285006    0.172003
285405    0.165401
285337    0.106924
285496    0.380085
285497    0.366275
285378    0.169366
285589    0.198928
285585    0.328937
285501    0.410137
285581    0.130904
            ...   
376367    0.169682
376366    0.269950
376362    0.694193
376363    0.705997
376365    0.169682
376364    0.269950
376228    0.393413
376265    0.253134
376286    0.792204
376320    0.268845
376314    0.235309
376327    0.829027
376385    0.792448
376435    0.972742
376370    0.973533
376434    0.430943
376459    0.440426
376478    0.104957
376473    0.278926
376484    0.173403
376482    0.220598
37